# Run Baysor on per-section CSVs

This notebook runs Baysor on each per-section CSV exported from `00-data-wrangling.ipynb`.
Update the configuration cell for your local Baysor environment and desired parameters.


In [1]:
using ProgressMeter
ProgressMeter.ijulia_behavior(:append)


IJuliaAppend::IJuliaBehavior = 2

In [2]:
import Pkg

# Update this to your Baysor environment path
baysor_env = "/Users/christoffer/Baysor"
Pkg.activate(baysor_env)

using Baysor
using DataFrames
using CSV

println("Threads: ", Threads.nthreads())


  Activating project at `~/Baysor`


Threads: 16


## Configuration


In [ ]:
# Input/output folders
input_dir = "/Volumes/processing2/nature-dev-mouse-reanalysis/data"
output_root = "/Volumes/processing2/nature-dev-mouse-reanalysis/data/baysor_output"

# Baysor parameters (adjust as needed)
baysor_params = (
    x_column = :x,
    y_column = :y,
    # For 2D data, set z_column = nothing
    gene_column = :gene,
    min_molecules_per_cell = 20,
    n_clusters = 3,
    scale = 31.0,
    iters = 500
)


(x_column = :x, y_column = :y, z_column = nothing, gene_column = :gene, min_molecules_per_cell = 20, n_clusters = 3, scale = 31.0, iters = 500)

## Discover input files


In [23]:
# Find all CSVs under input_dir (skip macOS AppleDouble files)
csv_files = String[]
for (root, _, files) in walkdir(input_dir)
    for f in files
        lf = lowercase(f)
        if startswith(f, "._")
            continue
        end
        if endswith(lf, ".csv")
            push!(csv_files, joinpath(root, f))
        end
    end
end

println("Found ", length(csv_files), " CSV files")
csv_files


Found 27 CSV files


27-element Vector{String}:
 "/Volumes/processing2/nature-dev-mouse-reanalysis/data/section_02A/spots.csv"
 "/Volumes/processing2/nature-dev-mouse-reanalysis/data/section_02B/spots.csv"
 "/Volumes/processing2/nature-dev-mouse-reanalysis/data/section_02C/spots.csv"
 "/Volumes/processing2/nature-dev-mouse-reanalysis/data/section_02D/spots.csv"
 "/Volumes/processing2/nature-dev-mouse-reanalysis/data/section_02E/spots.csv"
 "/Volumes/processing2/nature-dev-mouse-reanalysis/data/section_02F/spots.csv"
 "/Volumes/processing2/nature-dev-mouse-reanalysis/data/section_06A/spots.csv"
 "/Volumes/processing2/nature-dev-mouse-reanalysis/data/section_06B/spots.csv"
 "/Volumes/processing2/nature-dev-mouse-reanalysis/data/section_06C/spots.csv"
 "/Volumes/processing2/nature-dev-mouse-reanalysis/data/section_06D/spots.csv"
 "/Volumes/processing2/nature-dev-mouse-reanalysis/data/section_06E/spots.csv"
 "/Volumes/processing2/nature-dev-mouse-reanalysis/data/section_06F/spots.csv"
 "/Volumes/processing2/na

## Run Baysor


In [ ]:
"""
Check if a segmentation CSV already exists in the output folder.
"""
function segmentation_exists(output_dir::String)::Bool
    if !isdir(output_dir)
        return false
    end
    for f in readdir(output_dir)
        lf = lowercase(f)
        if occursin("segmentation", lf) && endswith(lf, ".csv")
            return true
        end
    end
    return false
end

"""
Run Baysor on a single CSV file.
"""
function run_baysor(input_path::String; params...)
    base = splitext(basename(input_path))[1]
    m = Int(params[:min_molecules_per_cell])
    s = params[:scale]
    s_str = Int(s) == s ? string(Int(s)) : string(s)
    output_dir = joinpath(dirname(input_path), "m$(m)_s$(s_str)")
    mkpath(output_dir)
    output_prefix = joinpath(output_dir, "segmentation")

    if segmentation_exists(output_dir)
        println("Skipping (already exists): ", input_path)
        return (status="skipped", output=output_dir)
    end

    iters = haskey(params, :iters) ? Int(params[:iters]) : 500
    cfg = Baysor.Utils.RunOptions()
    cfg.segmentation.iters = iters

    # Build kwargs without z_column unless provided
    kwargs = Dict(
        :x_column => params[:x_column],
        :y_column => params[:y_column],
        :gene_column => params[:gene_column],
        :min_molecules_per_cell => params[:min_molecules_per_cell],
        :n_clusters => params[:n_clusters],
        :scale => params[:scale],
        :output => output_prefix,
        :config => cfg,
    )

    if haskey(params, :z_column)
        kwargs[:z_column] = params[:z_column]
    end

    println("Running Baysor on: ", input_path)
    Baysor.CommandLine.run(input_path; kwargs...)

    return (status="success", output=output_dir)
end

results = []

for (i, file) in enumerate(csv_files)
    println("[", i, "/", length(csv_files), "]")
    try
        res = run_baysor(file; baysor_params...)
        push!(results, (file=file, status=res.status, output=res.output))
    catch e
        println("ERROR: ", e)
        push!(results, (file=file, status="error", error=string(e)))
    end
end

results


[1/27]
Running Baysor on: /Volumes/processing2/nature-dev-mouse-reanalysis/data/section_02A/spots.csv
[13:29:17] Info: Run R7172c4560
[13:29:17] Info: (2026-02-02) Run Baysor v0.7.1
[13:29:17] Info: Using local Baysor build
[13:29:17] Info: Loading data...
[13:29:17] Info: Loaded 58705 transcripts, 119 genes.
[13:29:18] Info: Estimating noise level
[13:29:18] Info: Done
[13:29:18] Info: Clustering molecules...


Progress:   0%|▏                                        |  ETA: 0:02:15
                   Iteration: 23
             Max. difference: 0.241
4m   Fraction of probs changed: 0.939


Progress:   0%|▏                                        |  ETA: 0:02:48
                   Iteration: 44
             Max. difference: 0.17
4m   Fraction of probs changed: 0.903


Progress:   1%|▎                                        |  ETA: 0:02:57
                   Iteration: 59
             Max. difference: 0.0725
4m   Fraction of probs changed: 0.858


Progress:   1%|▎                                        |  ETA: 0:03:03
                   Iteration: 74
             Max. difference: 0.0349
4m   Fraction of probs changed: 0.814


Progress:   1%|▍                                        |  ETA: 0:03:07
                   Iteration: 89
             Max. difference: 0.0764
4m   Fraction of probs changed: 0.761


Progress:   1%|▍                                        |  ETA: 0:03:10
                   It

[13:29:22] Info: Algorithm stopped after 198 iterations. Max. probability difference: 0.000405. Converged: true.
[13:29:22] Info: Done
[13:29:22] Info: Initializing algorithm. Scale: 31.0, scale std: 7.75, initial #components: 5870, #molecules: 58705.
[13:29:22] Info: Using the following additional information about molecules: [:confidence, :cluster]
[13:29:22] Info: Using 2D coordinates


Progress:   3%|█▏                                       |  ETA: 0:00:04
         Iteration: 13
    Noise level, %: 6.6
4m   Num. components: 873


Progress:   5%|██                                       |  ETA: 0:00:04
         Iteration: 25
    Noise level, %: 6.5
4m   Num. components: 979


Progress:   7%|███                                      |  ETA: 0:00:04
         Iteration: 36
    Noise level, %: 11.31
4m   Num. components: 960


Progress:  10%|████                                     |  ETA: 0:00:04
         Iteration: 49
    Noise level, %: 6.43
4m   Num. components: 1110


Progress:  13%|█████▍                                   |  ETA: 0:00:03
         Iteration: 66
    Noise level, %: 10.21
4m   Num. components: 1062


Progress:  16%|██████▊                                  |  ETA: 0:00:03
         Iteration: 82
    Noise level, %: 6.42
4m   Num. components: 1189


Progress:  20%|████████▎                                |  ETA: 0:00:03
         Iteration: 100
    Noise lev

[13:29:25] Info: Processing complete.
[13:29:25] Info: Estimating boundary polygons
[13:29:25] Info: Saving results to /Volumes/processing2/nature-dev-mouse-reanalysis/data/section_02A/m20_s31/segmentation
[13:29:25] Info: All done!
[2/27]
Running Baysor on: /Volumes/processing2/nature-dev-mouse-reanalysis/data/section_02B/spots.csv
[13:29:25] Info: Run R0b213d94e
[13:29:25] Info: (2026-02-02) Run Baysor v0.7.1
[13:29:25] Info: Using local Baysor build
[13:29:25] Info: Loading data...
[13:29:25] Info: Loaded 301410 transcripts, 119 genes.
[13:29:25] Info: Estimating noise level
[13:29:26] Info: Done
[13:29:27] Info: Clustering molecules...


...

Progress:   0%|▏                                        |  ETA: 0:00:55
         Iteration: 2
    Noise level, %: 3.25
4m   Num. components: 5528


Progress:   1%|▎                                        |  ETA: 0:01:02
         Iteration: 3
    Noise level, %: 28.4
4m   Num. components: 4188


Progress:   1%|▍                                        |  ETA: 0:01:09
         Iteration: 4
    Noise level, %: 4.46
4m   Num. components: 8857


Progress:   1%|▌                                        |  ETA: 0:01:03
         Iteration: 6
    Noise level, %: 23.26
4m   Num. components: 7313


Progress:   2%|▋                                        |  ETA: 0:01:04
         Iteration: 8
    Noise level, %: 3.17
4m   Num. components: 10963


Progress:   2%|▊                                        |  ETA: 0:01:03
         Iteration: 9
    Noise level, %: 19.74
4m   Num. components: 8954


Progress:   2%|▉                                        |  ETA: 0:00:57
         Iteration: 11
    Noise leve

[14:08:37] Info: Processing complete.
[14:08:38] Info: Estimating boundary polygons
[14:08:38] Info: Saving results to /Volumes/processing2/nature-dev-mouse-reanalysis/data/section_10B/m20_s31/segmentation


Progress: 100%|█████████████████████████████████████████| Time: 0:00:00


[14:08:40] Info: All done!
[15/27]
Running Baysor on: /Volumes/processing2/nature-dev-mouse-reanalysis/data/section_10C/spots.csv
[14:08:40] Info: Run R034b8610f
[14:08:40] Info: (2026-02-02) Run Baysor v0.7.1
[14:08:40] Info: Using local Baysor build
[14:08:40] Info: Loading data...
[14:08:40] Info: Loaded 1018032 transcripts, 119 genes.
[14:08:40] Info: Estimating noise level
[14:08:43] Info: Done
[14:08:45] Info: Clustering molecules...


Progress:   0%|                                         |  ETA: 1:10:34
                   Iteration: 2
             Max. difference: 0.594
4m   Fraction of probs changed: 0.975


Progress:   0%|                                         |  ETA: 1:11:44
                   Iteration: 3
             Max. difference: 0.672
4m   Fraction of probs changed: 0.975


Progress:   0%|                                         |  ETA: 1:09:37
                   Iteration: 4
             Max. difference: 0.659
4m   Fraction of probs changed: 0.975


Progress:   0%|                                         |  ETA: 1:09:31
                   Iteration: 5
             Max. difference: 0.59
4m   Fraction of probs changed: 0.974


Progress:   0%|                                         |  ETA: 1:10:10
                   Iteration: 6
             Max. difference: 0.575
4m   Fraction of probs changed: 0.974


Progress:   0%|                                         |  ETA: 1:10:38
                   Iteration: